In [1]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print('CUDA is not available.  Training on CPU ...')
else:
  print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch

# number of subprocesses to use for data loading
num_workers = 2
# how many samples per batch to load
batch_size = 128
# percentage of training set to use as validation
valid_size = 0.2

# Imbalance factor for CIFAR-100-LT
imbalance_factor = 100  # Set imbalance factor (e.g., 100, 50, 10)

# convert data to a normalized torch.FloatTensor
print('==> Preparing data..')

# Image augmentation is used to train the model
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),        # 이미지 크기 조정
    transforms.RandomHorizontalFlip(),           # 좌우 반전
    transforms.RandomRotation(15),               # 15도 범위 내에서 랜덤 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 밝기, 대비, 채도 변형
    transforms.RandomGrayscale(p=0.1),           # 이미지 10% 확률로 흑백 변환
    transforms.ToTensor(),                       # 텐서 변환
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),  # 정규화
])

# Only the data is normalized, no need to augment the test data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

# Choose the training and test datasets
train_data = datasets.CIFAR100('data', train=True,
                               download=True, transform=transform_train)
test_data = datasets.CIFAR100('data', train=False,
                              download=True, transform=transform_test)

# Function to create imbalanced CIFAR-100-LT dataset
def make_imbalanced(dataset, imbalance_factor):
    labels = np.array(dataset.targets)
    num_classes = len(np.unique(labels))
    
    # Create imbalance using exponential decay for class distribution
    class_counts = np.array([np.sum(labels == i) for i in range(num_classes)])
    max_class_samples = class_counts.max()
    
    # Calculate class sample counts using exponential decay
    class_sample_counts = [int(max_class_samples / (imbalance_factor ** (i / (num_classes - 1)))) for i in range(num_classes)]
    
    indices_to_keep = []
    
    # 각 클래스에 대해 남길 샘플 수를 설정하여 불균형하게 유지
    for label in range(num_classes):
        class_indices = np.where(labels == label)[0]
        np.random.shuffle(class_indices)
        
        # 클래스마다 남길 샘플 수 계산
        class_keep_count = class_sample_counts[label]
        indices_to_keep.extend(class_indices[:class_keep_count])
    
    return indices_to_keep

# Create imbalanced dataset by selecting a subset of CIFAR-100
train_indices = make_imbalanced(train_data, imbalance_factor)

# Function to create balanced validation set
def balanced_validation_split(dataset, train_indices, valid_size, num_classes):
    labels = np.array(dataset.targets)
    
    # Initialize lists to hold training and validation indices
    train_idx = []
    valid_idx = []
    
    # For each class, split according to valid_size
    for label in range(num_classes):
        class_indices = np.where(labels[train_indices] == label)[0]
        np.random.shuffle(class_indices)
        
        split = int(np.floor(valid_size * len(class_indices)))
        valid_idx.extend(class_indices[:split])  # Validation set
        train_idx.extend(class_indices[split:])  # Training set
    
    return train_idx, valid_idx

# Create balanced validation set
train_idx, valid_idx = balanced_validation_split(train_data, train_indices, valid_size, num_classes=100)

# Define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          num_workers=num_workers)

# CIFAR-100 classes (100개의 클래스 이름을 정의하지 않았지만 CIFAR-100 데이터셋은 100개의 클래스를 가짐)
print(f"Number of classes in CIFAR-100: {len(train_data.classes)}")

# Display the class distribution
train_labels = np.array(train_data.targets)
class_sample_count = np.array([np.sum(train_labels[train_indices] == i) for i in range(100)])

for i, count in enumerate(class_sample_count):
    print(f'Class {i}: {count} samples')


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Number of classes in CIFAR-100: 100
Class 0: 500 samples
Class 1: 477 samples
Class 2: 455 samples
Class 3: 434 samples
Class 4: 415 samples
Class 5: 396 samples
Class 6: 378 samples
Class 7: 361 samples
Class 8: 344 samples
Class 9: 328 samples
Class 10: 314 samples
Class 11: 299 samples
Class 12: 286 samples
Class 13: 273 samples
Class 14: 260 samples
Class 15: 248 samples
Class 16: 237 samples
Class 17: 226 samples
Class 18: 216 samples
Class 19: 206 samples
Class 20: 197 samples
Class 21: 188 samples
Class 22: 179 samples
Class 23: 171 samples
Class 24: 163 samples
Class 25: 156 samples
Class 26: 149 samples
Class 27: 142 samples
Class 28: 135 samples
Class 29: 129 samples
Class 30: 123 samples
Class 31: 118 samples
Class 32: 112 samples
Class 33: 107 samples
Class 34: 102 samples
Class 35: 98 samples
Class 36: 93 samples
Class 37: 89 samples
Class 38: 85 samples
Class 39: 81 samples
Cl

In [3]:
import torch
import torch.backends.cudnn as cudnn
import torchvision.models as models

# Pretrained ResNet101 모델 불러오기 (더 큰 teacher 모델)
Resnet18 = models.resnet18(pretrained=True)

# 분류할 클래스 수 수정 (예: CIFAR-10의 경우 10)
num_classes = 100

# 마지막 fully connected 레이어 변경
Resnet18.fc = torch.nn.Linear(Resnet18.fc.in_features, num_classes)

# GPU 사용 가능 여부 확인
train_on_gpu = torch.cuda.is_available()

# 모델이 GPU에서 학습되는 경우
if train_on_gpu:
    model = torch.nn.DataParallel(Resnet18)
    cudnn.benchmark = True
    model = model.cuda()

print(model)


/home/iai/anaconda3/envs/son/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iai/anaconda3/envs/son/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [4]:
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# ECE 계산 함수
def compute_ece(output, target, n_bins=15):
    softmaxes = F.softmax(output, dim=1)
    confidences, predictions = torch.max(softmaxes, 1)
    accuracies = predictions.eq(target)

    bin_boundaries = torch.linspace(0, 1, n_bins + 1)
    ece = torch.zeros(1, device=output.device)
    
    for bin_lower, bin_upper in zip(bin_boundaries[:-1], bin_boundaries[1:]):
        in_bin = confidences.gt(bin_lower.item()) * confidences.le(bin_upper.item())
        prop_in_bin = in_bin.float().mean()
        
        if prop_in_bin > 0:
            accuracy_in_bin = accuracies[in_bin].float().mean()
            avg_confidence_in_bin = confidences[in_bin].mean()
            ece += torch.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    return ece.item()

# Class Weights 계산 함수
def get_class_weights(train_loader, num_classes):
    class_count = torch.zeros(num_classes)
    for _, target in train_loader:
        class_count += torch.bincount(target, minlength=num_classes)
    
    total_count = class_count.sum().item()
    class_weights = total_count / (num_classes * class_count)  # 기본 가중치 계산
    return class_weights


# 학습 세팅
n_epochs = 200
valid_loss_min = np.Inf  # track change in validation loss

# 학습 데이터의 클래스 개수에 따라 가중치 계산
num_classes = 100  
class_weights = get_class_weights(train_loader, num_classes)

# GPU 사용 여부에 따라 class_weights를 GPU로 이동
if train_on_gpu:
    class_weights = class_weights.cuda()

# Cross-Entropy Loss에 클래스 가중치 적용 (학습용)
train_criterion = nn.CrossEntropyLoss(weight=class_weights)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

# Cosine Annealing Scheduler 사용
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)



for epoch in range(1, n_epochs+1):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0
    total = 0
    ece_sum = 0.0

    ###################
    # 모델 학습 #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = train_criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)

    ######################
    # 모델 검증 #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item() * data.size(0)
        
        # Accuracy 계산
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)

        # ECE 계산
        ece_sum += compute_ece(output, target)

    # 학습률 스케줄러 업데이트 (검증 손실을 인자로 전달)
    scheduler.step(valid_loss)

    # 평균 loss 및 accuracy 계산
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(valid_loader.sampler)
    valid_accuracy = 100 * correct / total
    valid_ece = ece_sum / len(valid_loader)

    # 학습 및 검증 통계 출력
    print(f'Epoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f} \tValidation Accuracy: {valid_accuracy:.2f}% \tValidation ECE: {valid_ece:.6f}')
    
    # 검증 손실이 감소할 경우 모델 저장
    if valid_loss <= valid_loss_min:
        print(f'Validation loss decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), 'ResNet18_cew.pt')
        valid_loss_min = valid_loss

# 최적 모델 로드
model.load_state_dict(torch.load('ResNet18_cew.pt'))


Epoch: 1 	Training Loss: 4.221192 	Validation Loss: 3.844777 	Validation Accuracy: 12.34% 	Validation ECE: 0.111398
Validation loss decreased (inf --> 3.844777). Saving model ...


/home/iai/anaconda3/envs/son/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 2 	Training Loss: 3.590713 	Validation Loss: 3.757828 	Validation Accuracy: 13.94% 	Validation ECE: 0.095058
Validation loss decreased (3.844777 --> 3.757828). Saving model ...
Epoch: 3 	Training Loss: 3.502636 	Validation Loss: 3.470225 	Validation Accuracy: 16.52% 	Validation ECE: 0.114682
Validation loss decreased (3.757828 --> 3.470225). Saving model ...
Epoch: 4 	Training Loss: 3.179080 	Validation Loss: 3.412645 	Validation Accuracy: 18.58% 	Validation ECE: 0.099955
Validation loss decreased (3.470225 --> 3.412645). Saving model ...
Epoch: 5 	Training Loss: 3.062292 	Validation Loss: 3.185612 	Validation Accuracy: 23.09% 	Validation ECE: 0.103786
Validation loss decreased (3.412645 --> 3.185612). Saving model ...
Epoch: 6 	Training Loss: 2.961229 	Validation Loss: 3.202824 	Validation Accuracy: 23.51% 	Validation ECE: 0.139925
Epoch: 7 	Training Loss: 2.929465 	Validation Loss: 3.155627 	Validation Accuracy: 25.15% 	Validation ECE: 0.121790
Validation loss decreased (3.185

/tmp/ipykernel_2047389/3695964090.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('ResNet18_cew.pt'))


<All keys matched successfully>

In [5]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import label_binarize

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))

ece_values = []
# 리스트 초기화 (F1 스코어 및 AUROC 계산용)
all_targets = []
all_predictions = []
all_probabilities = []

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    test_loss += loss.item() * data.size(0)
    
    # convert output probabilities to predicted class
    probabilities = F.softmax(output, dim=1)
    _, pred = torch.max(probabilities, 1)

    # ECE 계산
    ece = compute_ece(output, target)  # ECE 함수 사용
    ece_values.append(ece)

    # Accuracy 계산
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy()) if train_on_gpu else np.squeeze(correct_tensor.numpy())

    # Calculate test accuracy for each object class
    for i in range(len(data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

    # F1 스코어와 AUROC 계산을 위한 타겟 및 예측 값 저장
    all_targets.extend(target.cpu().detach().numpy())  # 실제 레이블 저장
    all_predictions.extend(pred.cpu().detach().numpy())  # 예측 레이블 저장
    all_probabilities.extend(probabilities.cpu().detach().numpy())  # 클래스별 확률 값 저장

# ECE 계산
overall_ece = sum(ece_values) / len(ece_values)
print(f'\nECE: {overall_ece:.4f}')

# Calculate and print overall test accuracy
test_accuracy = np.sum(class_correct) / np.sum(class_total)
print(f'\nTest Accuracy (Overall): {100. * test_accuracy:.2f}%')

# F1 Score 계산 (다중 클래스의 경우 평균 방식 지정 가능, 예: 'macro')
f1 = f1_score(all_targets, all_predictions, average='macro')
print(f'\nF1 Score (Macro): {f1:.4f}')

# AUROC 계산 (다중 클래스의 경우 각 클래스별 확률을 사용)
all_targets_onehot = label_binarize(all_targets, classes=list(range(100)))  # CIFAR-100에 맞게 0-9로 변환
all_probabilities = np.array(all_probabilities)  # 리스트를 NumPy 배열로 변환
auroc = roc_auc_score(all_targets_onehot, all_probabilities, multi_class='ovr')
print(f'AUROC (One-vs-Rest): {auroc:.4f}')



ECE: 0.1656

Test Accuracy (Overall): 41.91%

F1 Score (Macro): 0.4130
AUROC (One-vs-Rest): 0.9485
